# 이게 크롤링 하다가 지금 매매랑 전세 월세의 코드가 다르다.
# 이거 찾아는 보겠는데 혹시 ios 단에서 테스트 할 때 값 이상하면 말좀 해달라.
# 이거 매매 종류 볼때 매매랑 전세/월세 다르면 코드 다르게 돌리자
# 이거 지금 성수에 매매가 5개인데 하나만 달라서 이거만 따로 빼면 될듯

In [1]:
from selenium import webdriver
dabang_url ='https://www.dabangapp.com/search#/map?filters=%7B%22multi_room_type%22%3A%5B2%2C1%2C0%5D%2C%22selling_type%22%3A%5B0%2C1%2C2%5D%2C%22deposit_range%22%3A%5B0%2C999999%5D%2C%22price_range%22%3A%5B0%2C999999%5D%2C%22trade_range%22%3A%5B0%2C999999%5D%2C%22maintenance_cost_range%22%3A%5B0%2C999999%5D%2C%22include_maintenance_option1%22%3Atrue%2C%22room_size%22%3A%5B0%2C999999%5D%2C%22supply_space_range%22%3A%5B0%2C999999%5D%2C%22room_floor_multi%22%3A%5B1%2C2%2C3%2C4%2C7%2C6%2C5%5D%2C%22division%22%3Afalse%2C%22duplex%22%3Afalse%2C%22room_type%22%3A%5B1%2C2%5D%2C%22enter_date_range%22%3A%5B0%2C999999%5D%2C%22parking_average_range%22%3A%5B0%2C999999%5D%2C%22household_num_range%22%3A%5B0%2C999999%5D%2C%22parking%22%3Afalse%2C%22animal%22%3Afalse%2C%22short_lease%22%3Afalse%2C%22full_option%22%3Afalse%2C%22built_in%22%3Afalse%2C%22elevator%22%3Afalse%2C%22balcony%22%3Afalse%2C%22loan%22%3Afalse%2C%22safety%22%3Afalse%2C%22pano%22%3Afalse%2C%22deal_type%22%3A%5B0%2C1%5D%7D&position=%7B%22location%22%3A%5B%5B127.03782637531295%2C37.518247186167436%5D%2C%5B127.07787253873349%2C37.56435875152705%5D%5D%2C%22center%22%3A%5B127.05784328891325%2C37.541304716670695%5D%2C%22zoom%22%3A6%7D&search=%7B%22id%22%3A%2211200114%22%2C%22type%22%3A%22region%22%2C%22name%22%3A%22%EC%84%B1%EC%88%98%EB%8F%991%EA%B0%80%22%7D&tab=all'

## start staticmethod 
    - SocialLogin.start()
    - SalesForm.start()

In [2]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
driver.implicitly_wait(3)

In [3]:
driver.get(dabang_url)
seongsoo_1dong_detail_list = driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")

In [4]:
seongsoo_1dong_detail_list[0].get_attribute("href")

'https://www.dabangapp.com/room/5e61eb268c45fb4368c1a174'

In [5]:
bang_url_list = []

In [6]:
for url in seongsoo_1dong_detail_list:
    bang_url_list.append(url.get_attribute('href'))

In [7]:
# [driver.get(url) for url in bang_url_list]

# test code start

In [8]:
from selenium.common.exceptions import NoSuchElementException

In [9]:
driver.get(bang_url_list[0])

In [10]:
import time
time.sleep(5)
# 이거 안하면 아직 페이지 로딩 안되서 에러남 버튼 아직 없음

In [11]:
try:
    button = driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/div/div/button")
    driver.execute_script("arguments[0].click();", button)
except NoSuchElementException:
    pass    

In [12]:
unrefined_description = driver.find_elements_by_xpath("/html/body/div[1]/div/div[4]/div/div")

In [13]:
description = unrefined_description[0].get_attribute("innerText")

In [14]:
description

'서울숲역 도보7분거리,  2룸입니다.\nㅇ 2룸에 방하나는 크고 넓으며 도배해 드립니다.\n\nㅇ 대중교통 환승으로 2호선 뚝섬역 접근성이 좋습니다.\n\nㅇ 이마트가 도보 7분거리로 편리합니다.\n\nㅇ 서울숲, 한강이 인근에  있어 여가활동에 좋습니다.\n\nㅇ 메가박스가 도보 3분거리입니다.\n접기'

In [15]:
description.replace("\n", "")

'서울숲역 도보7분거리,  2룸입니다.ㅇ 2룸에 방하나는 크고 넓으며 도배해 드립니다.ㅇ 대중교통 환승으로 2호선 뚝섬역 접근성이 좋습니다.ㅇ 이마트가 도보 7분거리로 편리합니다.ㅇ 서울숲, 한강이 인근에  있어 여가활동에 좋습니다.ㅇ 메가박스가 도보 3분거리입니다.접기'

# 이건 데이터 어떻게 념겨줄건지 이야기 할 때 내가 안바꾸고 거기서 바꾸는게 더 편하지 않겠냐 물어보기 -- 위에 출력한 텍스트 두개

In [16]:
unrefined_address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div/div/p')
address = unrefined_address[0].get_attribute("innerText")
# 이건 테이블 분리 안하고 그냥 여기에 넣으면 될 듯 동 단위여서 의미가 없음
# 테이블 수정함 PostAddress
address

'서울특별시 성동구 성수동1가'

In [17]:
unrefined_salesform = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/div')
salesForm = unrefined_salesform[0].get_attribute("innerText")
salesForm

'전세 1억 2000\n만원'

# 매매 단계에서 억단위가 텍스트로 잡히는데 이게 모델링에선 인티저로 잡하서
# 크롤링 상황에서 에러 발생
# 문자형으로 하기에는 이후 필터링에서 어떻게 할 지가 아직 결정이 안되서 
# 우선 크롤링 확인용으로 데이터 타입은 문자형으로 해 두겠음


In [18]:
salesForm = salesForm.replace('/', ' ')

In [19]:
salesForm = salesForm.replace('\n', '')

In [20]:
salesForm = salesForm.split()

In [21]:
salesForm

['전세', '1억', '2000만원']

In [22]:
salesform_type = salesForm[0]

In [23]:
salesform_type

'전세'

In [24]:
salesform_deposit = salesForm[1]

In [25]:
salesform_deposit

'1억'

In [26]:
salesform_monthly = salesForm[2]

In [27]:
salesform_monthly
# SalesForm model의 salePrice라는 속성 지우기, 매매 가격도 deposit으로 통일 하는게 나을듯
# https://www.dabangapp.com/room/5dea1592bfd6ae3b8ca406ce 처럼 월세가 보증금마다 달라지는건 어떻게 할지 고민중

'2000만원'

In [28]:
real_pay = salesform_monthly.split('만')

In [29]:
real_pay = real_pay[0]

In [30]:
real_pay

'2000'

In [31]:
unrefined_floor = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[1]/div')

In [32]:
total_floor = unrefined_floor[0].get_attribute('innerText')

In [33]:
total_floor = total_floor.split('/')

In [34]:
floor = total_floor[0]

In [35]:
floor

'1층 '

In [36]:
totalFloor = total_floor[1]

In [37]:
totalFloor =totalFloor.replace(' ', '')

In [38]:
totalFloor

'3층'

In [39]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')

In [40]:
area = unrefined_area[0].get_attribute('innerText')

In [41]:
area

'42.97 / 46.28㎡'

In [42]:
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/button').click()

In [43]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')

In [44]:
supplyArea = unrefined_area[0].get_attribute('innerText')

In [45]:
supplyArea

'13 / 14평'

In [46]:
unrefined_shortRent = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[5]/p')

In [47]:
shortRent = unrefined_shortRent[0].get_attribute('innerText')

In [48]:
shortRent

'불가능'

In [49]:
# if shortRent=='불가능':
#     shortRent= False
# else:
#     shortRent= True

In [50]:
unrefined_management = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]')

In [51]:
unrefined_management = unrefined_management[0].get_attribute('innerText')

In [52]:
unrefined_management = unrefined_management.replace('\n', '')

In [53]:
unrefined_management

'없음'

In [54]:
unrefined_management = unrefined_management.replace(' ', '')

In [55]:
unrefined_management
# 이렇게 한번에 할지 아니면 아래처럼 분리할지 

'없음'

In [56]:
unrefined_management = unrefined_management.replace('(', ' ')

In [57]:
unrefined_management

'없음'

In [58]:
unrefined_management = unrefined_management.replace(')', ' ')

In [59]:
unrefined_management

'없음'

In [60]:
unrefined_management = unrefined_management.replace(',', ' ')

In [61]:
unrefined_management = unrefined_management.split(' ')

In [62]:
unrefined_management
# get_or_create()해야 할 듯 .

['없음']

In [63]:
managementPay = unrefined_management.pop(0)

In [64]:
managementPay

'없음'

In [65]:
totalPay = managementPay.split('만')

In [66]:
totalPay = totalPay[0]
# 이걸 MTM필드 totalFee로 한다. 속성으로 만들어서 필터링 할 때 조회 가능하게 고민.

In [67]:
unrefined_management

[]

## 이게 다방이 방 내놓을때에는 관리비 포함 항목들을 오브젝트로 만들어놨는데 
## 필터링할때에는 포함 항목들이 해당되지 않는다.
## 이게 다방은 없는데 우리는 있는 기능으로 만들 수 있긴 있을 것 같다.

In [68]:
unrefined_parking = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p')

In [69]:
unrefined_parking[0].get_attribute('innerText')
# 관리비는 따로 필터링 하지 않아서 그냥 텍스트로만 저장

'불가'

In [70]:
try:
    unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/label')
    unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/span')
except NoSuchElementException:
    pass
# 한달 생활비가 있는 글이 있고 없는 글이 있다.

In [71]:
unrefined_living_expenses= unrefined_living_expenses[0].get_attribute('innerText')
living_expenses = unrefined_living_expenses.replace(' ', '')
living_expenses

'0만원+α'

In [72]:
living_expenses_detail = unrefined_living_expenses_detail[0].get_attribute('innerText')
living_expenses_detail

'(-)'

In [73]:
unrefined_parking = driver.find_elements_by_xpath("/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p")
parking = unrefined_parking[0].get_attribute('innerText')
parking
# model에서 주차 비용 유무는 그냥 ios에서 보일때만 표시하고 django에 들어올때에는 없으면 안받고 있으면 받음.
# 주차비도 필터링에 조건 되는게 없으니까 따로 숫자 빼지는 않겠음.

'불가'

In [74]:
unrefined_moveIn = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[9]/div')
moveIn = unrefined_moveIn[0].get_attribute('innerText')
moveIn

'날짜 협의'

# 날짜도 데이터타임 필드인데 크롤링 시 문자형이 있어서 우선 이걸로 
# 이건 필드 두개 나눠서 크롤링 데이터는 문자형
# 생성하게 될 것들은 데이터타임필드로 하던 우선 보류

In [75]:
try:
    unrefined_option = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div[1]/div')
except NoSuchElementException:
    pass
unrefined_option = unrefined_option[0].get_attribute('innerText')
unrefined_option = unrefined_option.split('\n\n')
option = unrefined_option
option

['50m']

In [76]:
unrefined_heatingType = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[3]/div')
heatingType = unrefined_heatingType[0].get_attribute('innerText')
heatingType

'개별난방'

In [77]:
unrefined_pet = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[6]/div')
pet = unrefined_pet[0].get_attribute('innerText')
pet

'불가능'

In [78]:
unfined_elevator = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[5]/div')
elevator = unfined_elevator[0].get_attribute('innerText')
elevator

'없음'

### 복층, 1.5룸, 주방 분리형 아직 못찾아서 안넣었음

In [79]:
unrefined_builtIn = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[4]/div')
builtIn = unrefined_builtIn[0].get_attribute('innerText')
builtIn

'아님'

In [80]:
unrefined_veranda = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[7]/div')
veranda = unrefined_veranda[0].get_attribute('innerText')
veranda

'없음'

In [81]:
unrefined_depositLoan = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[8]/div')
depositLoan = unrefined_depositLoan[0].get_attribute('innerText')
depositLoan

'가능'

In [82]:
unrefined_securitySafety = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div[2]/div')
security = unrefined_securitySafety[0].get_attribute('innerText')
security= security.split('\n\n')
security

IndexError: list index out of range